In [1]:
import gym

In [2]:
env=gym.make('MountainCar-v0')

In [3]:
print(env.observation_space)
print(env.action_space)

Box(2,)
Discrete(3)


In [4]:
env.reset()
for _ in range(10):
    env.render()
    env.step(env.action_space.sample())
env.close()    

In [5]:
from collections import deque
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
import numpy as np
import random
import os

Using TensorFlow backend.


In [6]:
class Agent:
    def __init__(self,state_size,action_size):
        self.state_size=state_size
        self.action_size=action_size
        self.memory=deque(maxlen=2000)
        self.epsilon=1.0
        self.epsilon_decay=0.995
        self.epsilon_min=0.1
        self.gamma=0.95 ## Discount factor
        self.model = self._create_model()
    
    def _create_model(self):
        #neural network to approximate Q-learning Function
        model=Sequential()
        model.add(Dense(24,input_dim=self.state_size,activation='relu'))
        model.add(Dense(48,activation='tanh'))
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(optimizer=Adam(lr=0.001),loss='mse')
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done)) # remember previous experiences
        
    def act(self,state):
        if np.random.rand()<self.epsilon:
            return random.randrange(self.action_size)
        action = self.model.predict(state)
        return np.argmax(action[0])
    
    def train(self,batch_size=32):
        minibatch=random.sample(self.memory,batch_size)
        
        for state,action,reward,next_state,done in minibatch:
            if not done:
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target = reward
            
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            self.model.fit(state,target_f,epochs=1,verbose=0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def load(self,name):
        self.model.load_weights(name)
        
    def save(self,name):
        self.model.save_weights(name)                  

In [ ]:
n_episodes = 1000
#os.mkdir('MountainCar_model/')
output_dir = "MountainCar_model/"

state_size = 2
action_size =3
batch_size = 32

agent = Agent(state_size, action_size) # initialise agent
done = False
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])
    
    for time in range(5000):
        env.render()
        action = agent.act(state)
        next_state,reward,done,other_info = env.step(action) 
        reward = reward if not done else -10
        next_state = np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state,done)
        state = next_state
        
        if done:
            print("Game Episode :{}/{}, High Score:{},Exploration Rate:{:.2}".format(e,n_episodes,time,agent.epsilon))
            break
            
    if len(agent.memory)>batch_size:
        agent.train(batch_size)
    
    if e%50==0:
        agent.save(output_dir+"weights_"+'{:04d}'.format(e)+".hdf5")
        
env.close()

Game Episode :0/1000, High Score:199,Exploration Rate:1.0

Game Episode :1/1000, High Score:199,Exploration Rate:0.99
Game Episode :2/1000, High Score:199,Exploration Rate:0.99
Game Episode :3/1000, High Score:199,Exploration Rate:0.99
Game Episode :4/1000, High Score:199,Exploration Rate:0.98
Game Episode :5/1000, High Score:199,Exploration Rate:0.98
Game Episode :6/1000, High Score:199,Exploration Rate:0.97
Game Episode :7/1000, High Score:199,Exploration Rate:0.97
Game Episode :8/1000, High Score:199,Exploration Rate:0.96
Game Episode :9/1000, High Score:199,Exploration Rate:0.96
Game Episode :10/1000, High Score:199,Exploration Rate:0.95
Game Episode :11/1000, High Score:199,Exploration Rate:0.95
Game Episode :12/1000, High Score:199,Exploration Rate:0.94
Game Episode :13/1000, High Score:199,Exploration Rate:0.94
Game Episode :14/1000, High Score:199,Exploration Rate:0.93
Game Episode :15/1000, High Score:199,Exploration Rate:0.93
Game Episode :16/1000, High Score:199,Exploration 